# Actividad 4:
# Optimización y Análisis Geométrico de Funciones en Dos Variables

---

> **Importante:** Para el correcto uso del notebook, debe ejecutarse en el orden descrito más abajo. De lo contrario, pueden producirse errores debido a que algunas funciones aún no estarán definidas al momento de su uso.

---

## Estructura:
1. Decisiones de diseño.
2. Funciones definidas.
3. Visualización de resultados y gráficas.
4. Discusión sobre la relevancia en Machine Learning.

---

## Decisiones de diseño:

- Se utilizan las herramientas simbólicas de `SymPy` para el cálculo exacto de derivadas y análisis de la función.
- Se resuelven las ecuaciones simbólicas para hallar **uno o más puntos críticos** automáticamente, sin intervención manual.
- Se evalúa la matriz Hessiana en cada punto crítico para determinar su naturaleza (mínimo local, máximo local o punto de silla).
- Se emplea `Matplotlib` para la representación gráfica de la función y los puntos críticos, tanto en superficie 3D como en mapa de contorno 2D, para facilitar la interpretación visual.

---

## Funciones definidas

Este proyecto se estructura en cuatro funciones principales:

- **`definir_funcion()`**  
  Define la función **𝑔(𝑥,𝑦)=𝑥2+3𝑦2−4𝑥+2𝑦+1**, y calcula derivadas parciales, gradiente y matriz Hessiana.

- **`encontrar_puntos_criticos()`**  
  Resuelve el sistema **𝛻𝑔(𝑥,𝑦)=(0,0)** para hallar el/los punto(s) crítico(s).

- **`clasificar_punto_critico()`**  
  Evalúa la Hessiana en el punto crítico y clasifica su naturaleza según los valores propios.

- **`visualizar_funcion()`**  
  Genera gráficos 3D y de contorno, marcando el punto crítico y su clasificación.

---

> **Nota:** No es necesario importar Axes3D. Se puede habilitar proyección 3D directamente con projection='3d'.

---

In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # <- No es necesario, pero se deja para legibilidad

def definir_funcion():
    """
    Define la función simbólica g(x, y) = x^2 + 3y^2 - 4x + 2y + 1, y calcula sus derivadas parciales, 
    el gradiente y la matriz Hessiana.

    Returns:
        x (Symbol): Variable simbólica x.
        y (Symbol): Variable simbólica y.
        funcion (Expr): Expresión simbólica de g(x, y).
        derivada_x (Expr): Derivada parcial ∂g/∂x.
        derivada_y (Expr): Derivada parcial ∂g/∂y.
        gradiente (Matrix): Vector gradiente de g(x, y).
        hessiana (Matrix): Matriz Hessiana de g(x, y).
    """
    x, y = sp.symbols('x y')
    funcion = x**2 + 3*y**2 - 4*x + 2*y + 1

    derivada_x = sp.diff(funcion, x)
    derivada_y = sp.diff(funcion, y)
    gradiente = sp.Matrix([derivada_x, derivada_y])
    hessiana = sp.hessian(funcion, (x, y))

    return x, y, funcion, derivada_x, derivada_y, gradiente, hessiana

def encontrar_puntos_criticos(derivada_x, derivada_y, x, y):
    """
    Encuentra los puntos críticos de una función multivariable resolviendo el sistema:
    ∂g/∂x = 0 y ∂g/∂y = 0.

    Utiliza álgebra simbólica para encontrar soluciones exactas (si existen) del sistema
    de ecuaciones no lineales.

    Args:
        derivada_x (sympy.Expr): Derivada parcial de la función respecto a x (∂g/∂x).
        derivada_y (sympy.Expr): Derivada parcial de la función respecto a y (∂g/∂y).
        x (sympy.Symbol): Variable simbólica x.
        y (sympy.Symbol): Variable simbólica y.

    Returns:
        list[dict[sympy.Symbol, sympy.Expr]]: Lista de soluciones simbólicas del sistema,
        donde cada solución es un diccionario del tipo {x: valor, y: valor}.
        Si no se encuentran soluciones, se retorna una lista vacía.
    """
    soluciones = sp.solve([derivada_x, derivada_y], (x, y), dict=True)
    return soluciones

def clasificar_punto_critico(hessiana, x, y, punto):
    """
    Evalúa la matriz Hessiana en un punto crítico y clasifica el tipo de punto según sus valores propios.

    Args:
        hessiana (Matrix): Matriz Hessiana simbólica.
        x (Symbol): Variable simbólica x.
        y (Symbol): Variable simbólica y.
        punto (dict): Punto crítico {x: valor, y: valor}.

    Returns:
        clasificacion (str): Clasificación del punto ("mínimo local", "máximo local", "punto de silla").
        valores_propios (list): Lista de valores propios evaluados numéricamente.
    """
    hess_eval = hessiana.subs({x: punto[x], y: punto[y]})
    valores_propios = list(hess_eval.eigenvals().keys())

    if all(vp > 0 for vp in valores_propios):
        clasificacion = "mínimo local"
    elif all(vp < 0 for vp in valores_propios):
        clasificacion = "máximo local"
    else:
        clasificacion = "punto de silla"

    return clasificacion, valores_propios

def visualizar_funcion(funcion, x, y, puntos_criticos, clasificaciones):
    """
    Grafica la superficie 3D y el contorno 2D de la función g(x, y), destacando todos los puntos críticos.

    Args:
        funcion (Expr): Expresión simbólica de g(x, y).
        x (Symbol): Variable simbólica x.
        y (Symbol): Variable simbólica y.
        puntos_criticos (list): Lista de diccionarios con puntos críticos.
        clasificaciones (list): Lista de clasificaciones correspondientes a cada punto crítico.
    """
    funcion_numerica = sp.lambdify((x, y), funcion, modules='numpy')

    X = np.linspace(-5, 5, 100)
    Y = np.linspace(-5, 5, 100)
    XX, YY = np.meshgrid(X, Y)
    ZZ = funcion_numerica(XX, YY)

    fig = plt.figure(figsize=(12, 5))

    # Gráfico 3D
    ax1 = fig.add_subplot(1, 2, 1, projection='3d')
    ax1.plot_surface(XX, YY, ZZ, cmap='viridis', alpha=0.8)
    ax1.view_init(elev=30, azim=30)  # Cambia estos valores a gusto para rotación de mapa 3d

    # Mapa de contorno 2D
    ax2 = fig.add_subplot(1, 2, 2)
    contorno = ax2.contourf(XX, YY, ZZ, levels=30, cmap='viridis')

    for i, punto in enumerate(puntos_criticos):
        x_c = float(punto[x])
        y_c = float(punto[y])
        z_c = float(funcion_numerica(x_c, y_c))
        clasificacion = clasificaciones[i]

        ax1.scatter(x_c, y_c, z_c, color='red', s=60, label=f'Punto {i+1}: {clasificacion}')
        ax2.scatter(x_c, y_c, color='red', s=60, label=f'Punto {i+1}: {clasificacion}')

    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_zlabel('g(x, y)')
    ax1.set_title('Superficie de g(x, y)')
    ax1.legend()

    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    ax2.set_title('Mapa de contorno de g(x, y)')
    ax2.legend()
    plt.colorbar(contorno, ax=ax2)

    plt.tight_layout()
    plt.savefig('puntos_criticos.png')
    plt.show()

# Visualización de resultados y gráficas

Se aplican las funciones definidas y se muestran los resultados obtenidos.

---

In [ ]:
# Uso de las funciones definidas
x, y, funcion, derivada_x, derivada_y, gradiente, hessiana = definir_funcion()
puntos_criticos = encontrar_puntos_criticos(derivada_x, derivada_y, x, y)
clasificaciones = []

# Impresión de expresiones simbólicas
print("Función g(x, y):")
sp.pprint(funcion)
print("\nDerivadas parciales:")
print("∂g/∂x:")
sp.pprint(derivada_x)
print("∂g/∂y:")
sp.pprint(derivada_y)
print("\nGradiente ∇g(x, y):")
sp.pprint(gradiente)
print("\nMatriz Hessiana:")
sp.pprint(hessiana)

# Impresión de todos los puntos críticos encontrados
print("\n--- Puntos críticos encontrados ---")
for i, punto in enumerate(puntos_criticos):
    clasificacion, valores_propios = clasificar_punto_critico(hessiana, x, y, punto)
    clasificaciones.append(clasificacion)

    print(f"\nPunto crítico {i+1}:")
    print(f"Simbólico: {punto}")
    print(f"Numérico: (x = {float(punto[x])}, y = {float(punto[y])})")
    print(f"Tipo de punto: {clasificacion}")
    print("Valores propios de la Hessiana:", [float(v) for v in valores_propios])
    print(f"Valor de g(x, y) en este punto: {float(sp.lambdify((x, y), funcion)(punto[x], punto[y]))}")

# Visualización de superficie y mapa de contorno de g(x, y)
visualizar_funcion(funcion, x, y, puntos_criticos, clasificaciones)

---

# Discusión sobre Machine Learning

El cálculo del gradiente y la clasificación de puntos críticos es fundamental en Machine Learning, especialmente en algoritmos de optimización como el descenso de gradiente. Estos métodos buscan minimizar funciones de costo encontrando mínimos locales o globales, guiados por el gradiente.

La matriz Hessiana permite analizar la curvatura de la función y entender la naturaleza de los puntos críticos, lo que es clave para ajustar hiperparámetros y garantizar la convergencia de los modelos.

---

## Notas adicionales

- El resultado simbólico del o los puntos críticos se confirma gráficamente en los plots generados.
- El código actual es capaz de detectar y analizar automáticamente **uno o más puntos críticos**, evaluando su tipo según los valores propios de la matriz Hessiana.
- El uso de `SymPy` garantiza precisión analítica y evita errores de redondeo comunes en métodos numéricos.
- El gráfico guarda una imagen (`grafico_funcion_v2.png`) en la carpeta del proyecto como evidencia visual y apoyo al análisis simbólico.

---